In [ ]:
## The following can be loaded using the jupyter "magic" %load 
# %load ../helpers.py
%matplotlib inline
%env NI_PAGER="cat"
import pandas as pd
import numpy as np
from os.path import join, split, exists, splitext

In [ ]:
from shapely.geometry import box, shape
from shapely.affinity import scale
import shapely
import fiona
from pyproj import Proj, transform
from fiona.crs import from_epsg
from unidecode import unidecode
import itertools
import string

ONE_DEGREE_LATITUDE_IN_KM = 111.0
GEOCODING_DATA_FOLDER = "./raw_data"
OUTPUT_DATA_FOLDER = "./geohash_data"
COUNTRY_GH_PRECISION = 4
DMA_GH_PRECISION = 4
CITY_GH_PRECISION = 5
REGION_GH_PRECISION = 4
GEOHASH_ALPHABET='0123456789bcdefghjkmnpqrstuvwxyz'
OK_CHARACTERS = set(string.letters + '_')
NON_ARCTIC_WORLD = box(-180, -70, 180, 70)


OUTPUT_FN_TEMPLATE = '{}_{}s'

In [ ]:
def gh_bbox_size(size):
    long_precision = np.ceil(size/2.0)
    lat_precision = np.floor(size/2.0)
#     print long_precision, lat_precision
    long_step = 360.0/2**(long_precision)
    lat_step = 180.0/2**(lat_precision)
#     print long_step, lat_step
    return long_step, lat_step


def gh_bbox(point, size):
    long_step, lat_step = gh_bbox_size(size)
    point_long, point_lat = point
    long_min, lat_min = np.floor(point_long/long_step)*long_step, np.floor(point_lat/lat_step)*lat_step
    long_max, lat_max = np.ceil(point_long/long_step)*long_step, np.ceil(point_lat/lat_step)*lat_step
    return long_min, lat_min, long_max, lat_max


def make_grid(bounds, gh_size):
    long_min, lat_min, long_max, lat_max = bounds
    grid_top_left = gh_bbox((long_min, lat_min), gh_size)[:2]
    grid_bottom_right = gh_bbox((long_max, lat_max), gh_size)[2:]
    
    long_step, lat_step = gh_bbox_size(gh_size)
    longs = np.arange(grid_top_left[0], grid_bottom_right[0] + long_step, long_step)
    lats = np.arange(grid_top_left[1], grid_bottom_right[1] + lat_step, lat_step)
    long_pairs = zip(longs, longs[1:])
    lat_pairs = zip(lats, lats[1:])
    bounds = ((long_pair, lat_pair) for lat_pair, long_pair in itertools.product(lat_pairs, long_pairs))
    bboxes = [(long_min, lat_min, long_max, lat_max) 
              for (long_min, long_max), (lat_min, lat_max) in bounds ]
    return bboxes


def search_gh_levels(search_space, precisions, geo_buffered=False):
    output = []
    this_precision = precisions[0]
    if not search_space.bounds:
        return []
    ghs = make_grid(search_space.bounds, this_precision)
    for box_coords in ghs:
        coord_box = box(*box_coords)
        intersection = coord_box.intersection(search_space)
        centroid = tuple(coord_box.centroid.coords)[0]
        if intersection.area == coord_box.area:
            output.append((centroid, this_precision))
        elif intersection.area == 0:
            pass
        else:
            if len(precisions) > 1:
                higher_precision_ghs = search_gh_levels(intersection, 
                                                        precisions[1:],
                                                       geo_buffered)
                output += higher_precision_ghs
            elif not geo_buffered:  ## If there's no buffer, take an outer hull
                output.append((centroid, this_precision))
            else:  ## If the data has been buffered, only take an inner hull
                pass
    return output


def shape_to_geohashes(input_geometry, gh_precision, buffer_km=0, steps=3):
    if isinstance(input_geometry, shapely.geometry.Polygon):
        input_shapes = [input_geometry]
    else:
        input_shapes = input_geometry.geoms
        
    if buffer_km != 0:
        geo_buffer = True
        input_shapes = [buffer_shape(s, buffer_km) for s in input_shapes]
    else:
        geo_buffer = False
        

    points_and_precisions = []
    for g in input_shapes:
#         if not g.is_valid: g = g.buffer(0)
        assert g.is_valid
        precisions = [(gh_precision-i)*5 for i in range(steps - 1, -1, -1)]
        shape_point_precisions = search_gh_levels(g, precisions)
        points_and_precisions += shape_point_precisions
    return points_and_precisions




"""
Test code below. Check out gofreerange.geohash.com
"""
test_box_coords = (-11.25, 50.625, 0, 56.25)
test_box = box(*test_box_coords)
print test_box
print test_box.bounds
# grid = make_grid(test_box.bounds, 15)
print gh_bbox((-1, 53), 10)
print gh_bbox((-1, 53), 15)

In [ ]:
def default_process_properties(ps, _prop_names):
    clean_ps = [unidecode(p) if isinstance(p, basestring) else str(p) for p in ps]
    return clean_ps

def default_process_shape(s):
    return s

def shapefile_gh_run(input_fn, output_fn, properties, process_shape, 
                     preprocess_shape=default_process_shape,
                     process_properties=default_process_properties,
                     process_shape_kwargs={}, hadoop=False):
    if hadoop:
        !ni hdfst://{input_fn} \>input_file.tmp
        input_fn = 'input_file.tmp'
    count = 0
    with open('{}.tmp'.format(output_fn), 'w') as output_f:
        with fiona.open(input_fn, 'r') as source:
            for f in source:
                properties_dict = f['properties']
                raw_shape_properties = [properties_dict[prop] for prop in properties]
                if any(x is None for x in raw_shape_properties):
                    print properties_dict
                    print raw_shape_properties, "NULLS", "\n\n\n\n"
                    continue
                shape_properties = process_properties(raw_shape_properties, properties)
                properties_str = '\t'.join(shape_properties)
                if count %10 == 0: print properties_str
                
                this_shape = shape(f['geometry'])
                this_shape = preprocess_shape(this_shape)
                this_shape = this_shape.buffer(0)
                reasonable_shape = NON_ARCTIC_WORLD.intersection(this_shape)
                shape_data = process_shape(reasonable_shape, **process_shape_kwargs)
                if count % 10 == 0: print properties_str, len(shape_data)
                for shape_datum_str in shape_data:
                    output_f.write('{}\t{}\n'.format(shape_datum_str, properties_str))
                count += 1
    if 'precision' in process_shape_kwargs:
        postprocess_gh_data('{}.tmp'.format(output_fn), output_fn, 
                           process_shape_kwargs['precision'])
    !ni {output_fn}.tmp gABu \>{output_fn}
    return
              

def explore_shapefile(input_fn, n=3):
    with fiona.open(input_fn, 'r') as source:
        count = 0
        for f in source:
            properties_dict = f['properties']
            print properties_dict
            count += 1
            this_shape = shape(f['geometry'])
            print source.crs
            print f.crs
#             print this_shape
            if count > n: break

def geohash_cover(input_shape, precision, buffer_km=0):
    point_precisions = shape_to_geohashes(input_shape, precision, 
                                          buffer_km, steps=2)
    output_strs = ['{}\t{}\t{}'.format(lat, lon, precision) 
                   for (lon, lat), precision in point_precisions]
    return output_strs


def generate_centroid(input_shape):
    centroid = input_shape.centroid
    return ['{}\t{}'.format(centroid.y, centroid.x)]

In [ ]:
def postprocess_gh_data(input_fn, output_fn, gh_precision):
    !ni {input_fn} p'r ghe(a, b, c/5), FR 3' \>intermediate_gh.tmp
    with open('{}.tmp'.format(output_fn), 'w') as output_f:
        for line in open('intermediate_gh.tmp', 'r'):
            raw_gh = line.split('\t')[0]
            line_data = '\t'.join(line.split('\t')[1:])
            append_ghs = [GEOHASH_ALPHABET] * (gh_precision - len(raw_gh))
            output_ghs = itertools.product([raw_gh], *append_ghs)
            for output_gh in output_ghs:
                output_f.write('{}\t{}'.format(''.join(output_gh), line_data))                      
    return

In [ ]:
"""
Data Downloaded from: https://github.com/simzou/nielsen-dma/blob/master/nielsentopo.json
Then converted from topojson to shapefile.
"""

def process_dma_properties(ps, prop_names):
    return [process_dma_property(p, prop_name) for p, prop_name in zip(ps, prop_names)]

def process_dma_property(p, prop_name):
    fmt_str = 'geo__{}__{}'
    if prop_name == 'dma':
        return fmt_str.format('dma_id', p)
    else:
        while '(' in p:
            p = p[:p.index('(') -1] + p[p.index(')') + 1:]
        dma_pieces = p.split(', ')
        dma, state = dma_pieces[0], dma_pieces[-1]
        state = state[:2]
        short_dma = dma.split('-')[0]
        short_dma = short_dma.replace(' ', '_')
        short_dma = ''.join(ch for ch in short_dma if ch in OK_CHARACTERS)
        return fmt_str.format('dma', '{}_{}'.format(short_dma, state))

dma_output_fn = join(OUTPUT_DATA_FOLDER,
                     OUTPUT_FN_TEMPLATE.format('dma', 'gh{}'.format(DMA_GH_PRECISION)))
dma_input_fn = join(GEOCODING_DATA_FOLDER, 'nielsen_dma/nielsen_dma.shp')
dma_properties = ['dma1', 'dma']
dma_kwargs = {'precision': DMA_GH_PRECISION}

shapefile_gh_run(input_fn=dma_input_fn, 
                 output_fn=dma_output_fn, 
                 properties=dma_properties, 
                 process_shape=geohash_cover, 
                 process_properties=process_dma_properties,
                 process_shape_kwargs=dma_kwargs) 